Наша практика будет основана на соревновании Kaggle: Predicting a Biological Response (Прогнозирование биологического ответа).

Необходимо предсказать биологический ответ молекул (столбец 'Activity') по их химическому составу (столбцы D1-D1776).


Данные представлены в формате CSV.  Каждая строка представляет молекулу. 

Первый столбец Activity содержит экспериментальные данные, описывающие фактический биологический ответ [0, 1]; 
Остальные столбцы D1-D1776 представляют собой молекулярные дескрипторы — это вычисляемые свойства, которые могут фиксировать некоторые характеристики молекулы, например размер, форму или состав элементов.

### Оглавление

[0. Подготовка](#0.-Подготовка)
   * [0.1. Загрузка библиотек, подготовка данных](#0.1.-Загрузка-библиотек,-подготовка-данных)
   * [0.2. Логистическая регрессия и случайный лес - baselines](#0.2.-Логистическая-регрессия-и-случайный-лес---baselines)

[1. Поиск по сетке GridSearchCV](#1.-Поиск-по-сетке-GridSearchCV)
   * [1.1. Логистическая регрессия](#1.1.-Логистическая-регрессия)
   * [1.2. Cлучайный лес](#1.2.-Cлучайный-лес)

[2. Случайный поиск по сетке RandomizedSearchCV](#2.-Случайный-поиск-по-сетке-RandomizedSearchCV)
   * [2.1. Логистическая регрессия](#2.1.-Логистическая-регрессия)
   * [2.2. Cлучайный лес](#2.2.-Cлучайный-лес)
   
[3. Использование Hyperopt](#3.-Использование-Hyperopt)
   * [3.1. Логистическая регрессия](#3.1.-Логистическая-регрессия)
   * [3.2. Cлучайный лес](#3.2.-Cлучайный-лес)
   
[4. Использование Optuna](#4.-Использование-Optuna)
   * [4.1. Логистическая регрессия](#4.1.-Логистическая-регрессия)
   * [4.2. Cлучайный лес](#4.2.-Cлучайный-лес)

[5. Итого](#5.-Итого)

### 0. Подготовка

### 0.1. Загрузка библиотек, подготовка данных

In [1]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки
from sklearn.model_selection import GridSearchCV #поиск по сетке
from sklearn.model_selection import RandomizedSearchCV # случайный поиск по сетке

from sklearn.model_selection import cross_val_score #кросс-валидация
import hyperopt #библиотека продвинутого поиска гиперпараметров

# fmin - основная функция, она будет минимизировать наш функционал
# tpe - алгоритм оптимизации
# hp - включает набор методов для объявления пространства поиска гиперпараметров
# trails - используется для логирования результатов
from hyperopt import hp, fmin, tpe, Trials

#Optuna
import optuna

In [2]:
N_JOBS = 14 #16 ядер, 2 - для чего-то еще, пока модели считаются
RANDOM_STATE = 42

In [3]:
data = pd.read_csv('./data/_train_sem09 (1).csv')
print(data.shape)
display(data.head())

(3751, 1777)


,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# проверка на пропуски
print(f"всего пустых значений: {data.isnull().sum().sum()}")

всего пустых значений: 0


In [5]:
# проверка на нечисловые переменные
types = data.dtypes
cat_features = list(types[(types == 'object')].index)
if cat_features:
    print(cat_features)
else:
    print('все числовые')

все числовые


In [6]:
# разбиение на train / test
X = data.drop(['Activity'], axis=1)
y = data['Activity']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 1, test_size = 0.2)

### 0.2. Логистическая регрессия и случайный лес - baselines

Логистическая регрессия:

In [7]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(random_state=RANDOM_STATE, max_iter = 1000)

#Обучаем модель, минимизируя logloss
%time log_reg.fit(X_train, y_train)

y_test_log_pred = log_reg.predict(X_test)
print(f"Лог-регресия - accuracy на тестовом наборе: {log_reg.score(X_test, y_test):.2f}")
print(f"Лог-регресия - f1_score на тестовом наборе: {metrics.f1_score(y_test, y_test_log_pred):.2f}")

Wall time: 1.32 s
Лог-регресия - accuracy на тестовом наборе: 0.76
Лог-регресия - f1_score на тестовом наборе: 0.78


In [8]:
#Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(random_state=RANDOM_STATE)

#Обучаем модель
%time rf.fit(X_train, y_train)

y_test_rf_pred = rf.predict(X_test)
print(f"Случайный лес - accuracy на тестовом наборе: {rf.score(X_test, y_test):.2f}")
print(f"Случайный лес - f1_score на тестовом наборе: {metrics.f1_score(y_test, y_test_rf_pred):.2f}")

Wall time: 1.27 s
Случайный лес - accuracy на тестовом наборе: 0.79
Случайный лес - f1_score на тестовом наборе: 0.81


### 1. Поиск по сетке GridSearchCV

In [9]:
# сетка для логистической регрессии
param_grid_logreg = {
    'penalty': ['l2', 'none'], #тип регурялизации
    'solver': ['lbfgs', 'saga'], #алгоритм оптимизации
    'C': list(np.linspace(0.01, 1, 10, dtype=float))
    }

# сетка для случайного леса
param_grid_rf = {'n_estimators': list(range(50, 250, 50)),
              'min_samples_leaf': list(range(3, 6, 1)),
              'max_depth': list(np.linspace(7, 10, 4, dtype=int))
              }

### 1.1. Логистическая регрессия

In [10]:
grid_search_logreg = GridSearchCV(
    estimator=log_reg, 
    param_grid=param_grid_logreg, 
    cv=5, 
    n_jobs = N_JOBS
    )

%time grid_search_logreg.fit(X_train, y_train)

y_test_log_gridsearch_pred = grid_search_logreg.predict(X_test)
print(f"Лог-регресия - accuracy на тестовом наборе: {metrics.accuracy_score(y_test, y_test_log_gridsearch_pred):.2f}")
print(f"Лог-регресия - f1_score на тестовом наборе: {metrics.f1_score(y_test, y_test_log_gridsearch_pred):.2f}")
print(f"Наилучшие значения гиперпараметров: {grid_search_logreg.best_params_}")

Wall time: 6min 8s
Лог-регресия - accuracy на тестовом наборе: 0.76
Лог-регресия - f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'C': 0.12, 'penalty': 'l2', 'solver': 'lbfgs'}


### 1.2. Cлучайный лес

In [11]:
grid_search_rf = GridSearchCV(
    estimator=rf, 
    param_grid=param_grid_rf, 
    cv=5, 
    n_jobs = N_JOBS
    )

%time grid_search_rf.fit(X_train, y_train)

y_test_rf_gridsearch_pred = grid_search_rf.predict(X_test)
print(f"Случайный лес - accuracy на тестовом наборе: {metrics.accuracy_score(y_test, y_test_rf_gridsearch_pred):.2f}")
print(f"Случайный лес - f1_score на тестовом наборе: {metrics.f1_score(y_test, y_test_rf_gridsearch_pred):.2f}")
print(f"Наилучшие значения гиперпараметров: {grid_search_rf.best_params_}")

Wall time: 37.2 s
Случайный лес - accuracy на тестовом наборе: 0.79
Случайный лес - f1_score на тестовом наборе: 0.81
Наилучшие значения гиперпараметров: {'max_depth': 10, 'min_samples_leaf': 3, 'n_estimators': 200}


### 2. Случайный поиск по сетке RandomizedSearchCV

### 2.1. Логистическая регрессия

In [12]:
rand_grid_search_logreg = RandomizedSearchCV(
    estimator=log_reg, 
    param_distributions=param_grid_logreg, 
    cv=5,
    n_iter = 20, 
    n_jobs = N_JOBS
    )

%time rand_grid_search_logreg.fit(X_train, y_train)

y_test_log_rand_gridsearch_pred = rf.predict(X_test)
print(f"Лог-регресия - accuracy на тестовом наборе: {metrics.accuracy_score(y_test, y_test_log_rand_gridsearch_pred):.2f}")
print(f"Лог-регресия - f1_score на тестовом наборе: {metrics.f1_score(y_test, y_test_log_rand_gridsearch_pred):.2f}")
print(f"Наилучшие значения гиперпараметров: {rand_grid_search_logreg.best_params_}")

Wall time: 3min 19s
Лог-регресия - accuracy на тестовом наборе: 0.79
Лог-регресия - f1_score на тестовом наборе: 0.81
Наилучшие значения гиперпараметров: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.12}


### 2.2. Cлучайный лес

In [13]:
rand_grid_search_rf = RandomizedSearchCV(
    estimator=rf, 
    param_distributions=param_grid_rf, 
    cv=5,
    n_iter = 10, 
    n_jobs = N_JOBS
    )

%time rand_grid_search_rf.fit(X_train, y_train)

y_test_rf_rand_gridsearch_pred = rf.predict(X_test)
print(f"Случайный лес - accuracy на тестовом наборе: {metrics.accuracy_score(y_test, y_test_rf_rand_gridsearch_pred):.2f}")
print(f"Случайный лес - f1_score на тестовом наборе: {metrics.f1_score(y_test, y_test_rf_rand_gridsearch_pred):.2f}")
print(f"Наилучшие значения гиперпараметров: {rand_grid_search_rf.best_params_}")

Wall time: 9.71 s
Случайный лес - accuracy на тестовом наборе: 0.79
Случайный лес - f1_score на тестовом наборе: 0.81
Наилучшие значения гиперпараметров: {'n_estimators': 100, 'min_samples_leaf': 3, 'max_depth': 10}


### 3. Использование Hyperopt

### 3.1. Логистическая регрессия

In [14]:
list_penalty=['l2', 'none']
list_solver=['lbfgs', 'saga']

space_hyperopt_logreg = {
    'penalty': hp.choice('penalty', list_penalty), #тип регурялизации
    'solver': hp.choice('solver', list_solver), #алгоритм оптимизации
    'C': hp.uniform('C', 0.01, 1)
    }

In [15]:
def hyperopt_log(params, cv=5, X=X_train, y=y_train, random_state=RANDOM_STATE):
    # используем эту комбинацию для построения модели
    params = {'penalty': params['penalty'],
              'solver': params['solver'],
              'C': params['C']
             }
    
    model = linear_model.LogisticRegression(**params, random_state=RANDOM_STATE, max_iter=1000)

    # обучаем модель
    #model.fit(X, y)
    #score = metrics.f1_score(y, model.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=N_JOBS).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [16]:
# начинаем подбор гиперпараметров
trials_logreg = Trials() # используется для логирования результатов

best_logreg=fmin(hyperopt_log, # наша функция 
                 space=space_hyperopt_logreg, # пространство гиперпараметров
                 algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
                 max_evals=30, # максимальное количество итераций
                 trials=trials_logreg, # логирование результатов
                 rstate=np.random.default_rng(RANDOM_STATE)# фиксируем для повторяемости результата
                 )
print("Наилучшие значения гиперпараметров {}".format(best_logreg))

100%|███████████████████████████████████████████████| 30/30 [06:19<00:00, 12.65s/trial, best loss: -0.7838505949712216]
Наилучшие значения гиперпараметров {'C': 0.02856439121520049, 'penalty': 0, 'solver': 0}


In [17]:
# рассчитаем точность для тестовой выборки
model_hyperopt_logreg = linear_model.LogisticRegression(
    random_state=RANDOM_STATE, 
    C=best_logreg['C'],
    penalty=list_penalty[best_logreg['penalty']],
    solver=list_solver[best_logreg['solver']],
    max_iter=1000
    )

model_hyperopt_logreg.fit(X_train, y_train)

print(f"лучшие параметры: C = {best_logreg['C']}, penalty = {list_penalty[best_logreg['penalty']]}, \
solver = {list_solver[best_logreg['solver']]}"
    )

y_test_hyperopt_logreg_pred = model_hyperopt_logreg.predict(X_test)
print(f"Лог-регресия - accuracy на тестовом наборе: {metrics.accuracy_score(y_test, y_test_hyperopt_logreg_pred):.2f}")
print(f"Лог-регресия - f1_score на тестовом наборе: {metrics.f1_score(y_test, y_test_hyperopt_logreg_pred):.2f}")

лучшие параметры: C = 0.02856439121520049, penalty = l2, solver = lbfgs
Лог-регресия - accuracy на тестовом наборе: 0.77
Лог-регресия - f1_score на тестовом наборе: 0.80


### 3.2. Cлучайный лес

In [18]:
# зададим пространство поиска гиперпараметров
space_hyperopt_rf={
    'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
    'max_depth' : hp.quniform('max_depth', 15, 26, 1),
    'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
    }

In [19]:
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=RANDOM_STATE):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=RANDOM_STATE)

    # обучаем модель
    #model.fit(X, y)
    #score = metrics.f1_score(y, model.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=N_JOBS).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [20]:
%%time

trials_rf = Trials() # используется для логирования результатов

best_rf=fmin(hyperopt_rf, # наша функция 
          space=space_hyperopt_rf, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials_rf, # логирование результатов
          rstate=np.random.default_rng(RANDOM_STATE) # фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best_rf))

100%|███████████████████████████████████████████████| 20/20 [00:36<00:00,  1.84s/trial, best loss: -0.8080577050312506]
Наилучшие значения гиперпараметров {'max_depth': 18.0, 'min_samples_leaf': 2.0, 'n_estimators': 103.0}
Wall time: 36.8 s


In [21]:
# рассчитаем точность для тестовой выборки
model_hyperopt_rf = ensemble.RandomForestClassifier(
    random_state=RANDOM_STATE, 
    max_depth=int(best_rf['max_depth']),
    min_samples_leaf=int(best_rf['min_samples_leaf']),
    n_estimators=int(best_rf['n_estimators'])
    )

model_hyperopt_rf.fit(X_train, y_train)

print(f"лучшие параметры: {best_rf}")

y_test_hyperopt_rf_pred = model_hyperopt_rf.predict(X_test)
print(f"Лог-регресия - accuracy на тестовом наборе: {metrics.accuracy_score(y_test, y_test_hyperopt_rf_pred):.2f}")
print(f"Лог-регресия - f1_score на тестовом наборе: {metrics.f1_score(y_test, y_test_hyperopt_rf_pred):.2f}")

лучшие параметры: {'max_depth': 18.0, 'min_samples_leaf': 2.0, 'n_estimators': 103.0}
Лог-регресия - accuracy на тестовом наборе: 0.80
Лог-регресия - f1_score на тестовом наборе: 0.82


### 4. Использование Optuna

### 4.1. Логистическая регрессия

In [22]:
def optuna_log(trial):
    # задаем пространства поиска гиперпараметров
    penalty = trial.suggest_categorical('penalty', list_penalty)
    solver = trial.suggest_categorical('solver', list_solver)
    C = trial.suggest_float('C', 0.01, 1)

    # создаем модель
    model = linear_model.LogisticRegression(
        penalty=penalty,
        solver=solver,
        random_state=RANDOM_STATE,
        max_iter=1000)
        
    # обучаем модель
    #model.fit(X_train_scaled, y_train)
    #score = metrics.f1_score(y_train, model.predict(X_train_scaled))
    
    cv = 5
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=N_JOBS).mean()

    return score

In [23]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study_log = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study_log.optimize(optuna_log, n_trials=30)

# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study_log.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study_log.best_value))

[I 2023-04-22 15:54:02,575] A new study created in memory with name: LogisticRegression
[I 2023-04-22 15:54:19,564] Trial 0 finished with value: 0.7233925031705929 and parameters: {'penalty': 'none', 'solver': 'lbfgs', 'C': 0.10309234953534055}. Best is trial 0 with value: 0.7233925031705929.
[I 2023-04-22 15:55:02,050] Trial 1 finished with value: 0.7567875582638045 and parameters: {'penalty': 'none', 'solver': 'saga', 'C': 0.3600378388507649}. Best is trial 1 with value: 0.7567875582638045.
[I 2023-04-22 15:55:19,021] Trial 2 finished with value: 0.7233925031705929 and parameters: {'penalty': 'none', 'solver': 'lbfgs', 'C': 0.8418164504879109}. Best is trial 1 with value: 0.7567875582638045.
[I 2023-04-22 15:55:49,177] Trial 3 finished with value: 0.7749759884359712 and parameters: {'penalty': 'l2', 'solver': 'saga', 'C': 0.9219114714637002}. Best is trial 3 with value: 0.7749759884359712.
[I 2023-04-22 15:56:06,010] Trial 4 finished with value: 0.7233925031705929 and parameters: {'p

Наилучшие значения гиперпараметров {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.9136317933243497}
f1_score на обучающем наборе: 0.78
Wall time: 13min 42s


In [24]:
# рассчитаем точность для тестовой выборки
model_optuna_log = linear_model.LogisticRegression(**study_log.best_params,random_state=RANDOM_STATE)
model_optuna_log.fit(X_train, y_train)
y_train_optuna_log_pred = model_optuna_log.predict(X_train)
print("accuracy на тестовом наборе: {:.2f}".format(model_optuna_log.score(X_test, y_test)))
y_test_optuna_log_pred = model_optuna_log.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test_optuna_log_pred, y_test)))

accuracy на тестовом наборе: 0.76
f1_score на тестовом наборе: 0.78


C:\Users\micha\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### 3.2. Cлучайный лес

In [25]:
def optuna_rf(trial):
    # задаем пространства поиска гиперпараметров
    n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
    max_depth = trial.suggest_int('max_depth', 10, 30, 1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1) 

    # создаем модель
    model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                            max_depth=max_depth,
                                            min_samples_leaf=min_samples_leaf,
                                            random_state=RANDOM_STATE)
    # обучаем модель
    #model.fit(X_train_scaled, y_train)
    #score = metrics.f1_score(y_train, model.predict(X_train_scaled))
    
    cv = 5
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=N_JOBS).mean()

    return score

In [26]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study_rf = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study_rf.optimize(optuna_rf, n_trials=30)

# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study_rf.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study_rf.best_value))

[I 2023-04-22 16:07:46,003] A new study created in memory with name: RandomForestClassifier
[I 2023-04-22 16:07:49,376] Trial 0 finished with value: 0.8026287133558003 and parameters: {'n_estimators': 199, 'max_depth': 21, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.8026287133558003.
[I 2023-04-22 16:07:51,751] Trial 1 finished with value: 0.7966918913204883 and parameters: {'n_estimators': 142, 'max_depth': 11, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.8026287133558003.
[I 2023-04-22 16:07:53,827] Trial 2 finished with value: 0.8046600169262316 and parameters: {'n_estimators': 109, 'max_depth': 23, 'min_samples_leaf': 7}. Best is trial 2 with value: 0.8046600169262316.
[I 2023-04-22 16:07:56,706] Trial 3 finished with value: 0.7991830434433653 and parameters: {'n_estimators': 149, 'max_depth': 25, 'min_samples_leaf': 8}. Best is trial 2 with value: 0.8046600169262316.
[I 2023-04-22 16:07:59,246] Trial 4 finished with value: 0.8051061966550603 and parameters: {'n_

Наилучшие значения гиперпараметров {'n_estimators': 194, 'max_depth': 30, 'min_samples_leaf': 2}
f1_score на обучающем наборе: 0.82
Wall time: 1min 32s


In [27]:
# рассчитаем точность для тестовой выборки
model_optuna_rf = ensemble.RandomForestClassifier(**study_rf.best_params,random_state=RANDOM_STATE)
model_optuna_rf.fit(X_train, y_train)
y_train_optuna_rf_pred = model_optuna_rf.predict(X_train)
print("accuracy на тестовом наборе: {:.2f}".format(model_optuna_rf.score(X_test, y_test)))
y_test_optuna_rf_pred = model_optuna_rf.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test_optuna_rf_pred, y_test)))

accuracy на тестовом наборе: 0.80
f1_score на тестовом наборе: 0.82


#### 5. Итого

Методы хорошие, полезные и интересные. 

Logistic на таких количествах признаков считается неприлично долго. Лес вырастить, понятное дело, быстрее.

Optuna понравилась по удобству - проще Hyperopt, да и вроде как побыстрее будет.
Результаты похожи с точностью до случайных колебаний и округлений.

Если есть время, можно делать полный Grid search. Чем больше вариантов, тем лучше.

А вообще пришлось еще почитать, т.к. не все необходимое было дано в курсе.